In [ ]:
import os
import cv2
import numpy as np
import datetime as dt
import face_recognition as fr

import csv

In [ ]:
tym = dt.datetime.now()

In [ ]:
path = 'images'

my_list = os.listdir(path)
my_list

In [ ]:
imgs = []
clsnames = []

for i in my_list :

    imgpath = os.path.join(path, i)
    current_img = cv2.imread(imgpath)

    imgs.append(current_img)
    clsnames.append(i.split('.')[0])

In [ ]:
def encode_faces(img) :

    encoded_list = []

    for i in imgs :

        img = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)

        face_loc = fr.face_locations(img)
        face_enc = fr.face_encodings(img, face_loc)[0]

        encoded_list.append(face_enc)

    return encoded_list

In [ ]:
trained_imgs = encode_faces(imgs)

In [ ]:
vdo = cv2.VideoCapture(0)

while True :

    succ, img = vdo.read()
    if not succ:
        break
    img1 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    face_in_frame = fr.face_locations(img1)
    face_encoder = fr.face_encodings(img1, face_in_frame)

    for encoder, faceloc in zip(face_encoder, face_in_frame) :

        match = fr.compare_faces(trained_imgs, encoder)

        distance = fr.face_distance(trained_imgs, encoder)
        index = np.argmin(distance)

        if match[index]:
            name = clsnames[index]
            
            # Check if already marked for today
            today = dt.date.today().isoformat()
            csv_file = 'attendance.csv'
            already_present = False
            try:
                with open(csv_file, 'r') as f:
                    reader = csv.reader(f)
                    for row in reader:
                        if len(row) >= 2 and row[0] == name and row[1] == today:
                            already_present = True
                            break
            except FileNotFoundError:
                pass  # File doesn't exist, so not present
            
            if not already_present:
                with open(csv_file, 'a', newline='') as f:
                    writer = csv.writer(f)
                    now = dt.datetime.now()
                    writer.writerow([name, today, now.strftime('%H:%M:%S')])
                print(f"Attendance marked for {name}")
            else:
                print(f"{name} already marked today")
        else :
            name = 'Unknown'
            
        # Draw rectangle and name
        y1, x2, y2, x1 = faceloc
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
       
    cv2.imshow('Attendance', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

vdo.release()
cv2.destroyAllWindows()